In [6]:
# Скачиваем датасет с Hugging Face
import requests

url = "https://huggingface.co/CCRss/test/resolve/main/Housing.csv"
r = requests.get(url)

with open("Housing.csv", "wb") as f:
    f.write(r.content)

In [2]:
# Импорты
import pandas as pd
import os
import random
import textwrap
from openai import OpenAI
from dotenv import load_dotenv

In [3]:
# Объявляем наш датасет
housing_data = pd.read_csv('Housing.csv')
data = housing_data.to_dict(orient='records')
housing_data.head()
#housing_data.any()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
# Объявление фунций

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

client = OpenAI(api_key=api_key)

def generate_description(entry):
    """Генерирует описание недвижимости с использованием модели OpenAI."""
    prompt = f"Create a catchy Instagram ad description for a house with the following details:\nPrice: {entry['price']}\nArea: {entry['area']} sq ft\nBedrooms: {entry['bedrooms']}\nBathrooms: {entry['bathrooms']}\nStories: {entry['stories']}\nMain Road: {entry['mainroad']}\nGuestroom: {entry['guestroom']}\nBasement: {entry['basement']}\nHot Water Heating: {entry['hotwaterheating']}\nAir Conditioning: {entry['airconditioning']}\nParking: {entry['parking']}\nPreferred Area: {entry['prefarea']}\nFurnishing Status: {entry['furnishingstatus']}"

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0.2,
        max_tokens=150
    )

    return response.choices[0].message.content

def generate_image(prompt):
    """Генерирует изображение с использованием модели Dall-E."""
    try:
        response = client.images.generate(
            model="dall-e-3",
            prompt=prompt,
            size="1024x1024",
            quality="standard",
            n=1,
            style='natural' # or vivid если нужно hyper-real and dramatic images
        )

        return response.data[0].url
    except Exception as e:
        print(f"Произошла ошибка при генерации изображения: {e}")
        return None

def append_to_file(file_name, text):
    """Добавляет текст в файл."""
    with open(file_name, 'a', encoding='utf-8') as file:
        file.write(text + "\n")

def pretty_print(label, text, width=80):
    """Выводит текст с заданным форматированием."""
    print(f"{label}:")
    print(textwrap.fill(text, width=width))
    print("\n" + "-" * width + "\n")

def format_input_data(entry):
    """Форматирует входные данные"""
    return (
        "\n" + "-" * 80 + "\n" +
        f"Price: {entry['price']}\n"
        f"Area: {entry['area']} sq ft\n"
        f"Bedrooms: {entry['bedrooms']}\n"
        f"Bathrooms: {entry['bathrooms']}\n"
        f"Stories: {entry['stories']}\n"
        f"Main Road: {entry['mainroad']}\n"
        f"Guestroom: {entry['guestroom']}\n"
        f"Basement: {entry['basement']}\n"
        f"Hot Water Heating: {entry['hotwaterheating']}\n"
        f"Air Conditioning: {entry['airconditioning']}\n"
        f"Parking: {entry['parking']}\n"
        f"Preferred Area: {entry['prefarea']}\n"
        f"Furnishing Status: {entry['furnishingstatus']}"
        "\n" + "-" * 80 + "\n"
    )

In [5]:
# Генерация описания

# Получаем случайные записи
random_entry = random.choice(data)
input_data = format_input_data(random_entry)
print(input_data)

# Генерируем
description = generate_description(random_entry)
pretty_print("Generated Description", description)

# Формирование текста для записи в файл
output_text = input_data + "\nGenerated Description:\n" + description + "\n" + "-" * 80 + "\n"

# Запись в файл EXAMPLE.md
print('Описание добавлено в EXAMPLE.md ✅')
append_to_file('EXAMPLE.md', output_text)


--------------------------------------------------------------------------------
Price: 9310000
Area: 6550 sq ft
Bedrooms: 4
Bathrooms: 2
Stories: 2
Main Road: yes
Guestroom: no
Basement: no
Hot Water Heating: no
Air Conditioning: yes
Parking: 1
Preferred Area: yes
Furnishing Status: semi-furnished
--------------------------------------------------------------------------------

Generated Description:
"🏡 Dreamy and spacious 4-bedroom, 2-bathroom house with 6550 sq ft of pure
luxury! ✨✨ Enjoy the comfort of 2 stories, air conditioning, and a main road
location. 🛣️💨 This semi-furnished gem is perfect for those seeking a stylish and
cozy home. 💫💕 Don't miss out on this incredible opportunity for just $9,310,000!
💰🔑 Book your viewing now! 📲🏡 #DreamHome #LuxuryLiving #HomeSweetHome"

--------------------------------------------------------------------------------

Описание добавлено в EXAMPLE.md ✅


In [27]:
# Генерация изображений
#Разные промпты для генерация изображений
luxurious = "Generate an image of a luxurious real estate property, highlighting features like a grand entrance and designer interiors, for a high-end Instagram real estate account ad, using data from the Housing Prices Dataset."
realistic = "Create a realistic image of a real estate property, focusing on accurate architectural details and surrounding environment, for use in an Instagram ad, based on the Housing Prices Dataset."
shortest = "Generate a bright, clear image of a modern real estate property, suitable for an Instagram ad."
longest = "Create a highly detailed and vivid image of a real estate property, featuring modern architecture, a lush garden, and a cozy outdoor space. The image should be high-resolution, with a color palette that is bright and inviting, accurately representing the property's size and design. Include subtle branding elements for an Instagram real estate account, using the OpenAI ChatGPT gpt3.5-turbo API and data from the Housing Prices Dataset."

#Генерация изображения
image_url = generate_image(f"{shortest}")
print(f"Generated Image URL: {image_url}")

Generated Image URL: https://oaidalleapiprodscus.blob.core.windows.net/private/org-Akry343c8Cp6ycMk8CkWJK4Y/user-7qPIMhAFwBK8o2yNv8VTOYub/img-q4yMkw0XxRdwq9x7LppoZGma.png?st=2023-11-30T14%3A53%3A04Z&se=2023-11-30T16%3A53%3A04Z&sp=r&sv=2021-08-06&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2023-11-29T19%3A09%3A26Z&ske=2023-11-30T19%3A09%3A26Z&sks=b&skv=2021-08-06&sig=5UDuhbHQru/8KkuctgyCKg/LOTlZ/fDuD67O7bgkT0U%3D


In [26]:
## Создание EXAMPLE.txt (20 примеров)
# def write_descriptions_to_file(data, file_name, num_examples=20):
#     """Записывает описания в файл."""
#     with open(file_name, 'w', encoding='utf-8') as file:
#         for i in range(1, num_examples + 1):
#             random_entry = random.choice(data)
#             input_data = format_input_data(random_entry)
#             description = generate_description(random_entry)

#             file.write(f"Example {i}\n")
#             file.write(input_data)
#             file.write("Generated Description:\n")
#             file.write(textwrap.fill(description, width=80))
#             file.write("\n" + "-" * 80 + "\n\n")

# write_descriptions_to_file(data, 'EXAMPLE.txt')
